# *3. SQL* 

В базе данных вашей компании присутствуют следующие таблицы

* city
    * city_id – id города, где проходит акция
    * client_city_id – id города, где находится покупатель
    * city – город
    * client_city – город, где находится покупатель
* client
    * client_id – id покупателя
    * client_city_id – id города, где находится покупатель
    * birth_date – дата рождения покупателя
    * registration – дата регистрации покупателя
* promotion
    * promotion_id – id акции
    * category_id – id категории
    * promotion_name – акция
    * category_name – категория
    * partner_id – id партнёра
    * partner_name – партнёр
* purchase
    * purchase_id – id покупки
    * partner_id – id партнёра
    * client_id – id покупателя
    * city_id – id города
    * promotion_id – id акции
    * category_id – id категории
    * purchase_date – дата покупки
    * price – цена за единицу товара
    * quantity – число проданных единиц
    * status – статус покупки
    
Вам нужно написать запрос, чтобы получить такую таблицу

* purchase_date – дата покупки
* purchase_id – id покупки
* client_id – id покупателя
* client_age – возраст покупателя
* client_registration_age – как долго человек пользуется вашими магазинами
* client_category – содержит new или old, если это первая или последующая покупка соответственно
* promotion_name – акция
* category_name – категория
* partner_name – партнёр
* client_city – город, где находится покупатель
* city – город
* revenue – сумма выручки
* quantity – число проданных единиц

При этом в таблице должны присутствовать только значения, где в поле status стоит значение 1, и только те, где purchase_date находится в диапазоне от 01.05.2020 до 01.08.2020

In [1]:
import pandas as pd
import pandahouse as ph

In [2]:
connection_default = {'host': 'http://clickhouse.beslan.pro:8080',
                      'database':'default',
                      'user':'student', 
                      'password':'dpo_python_2020'
                     }

In [4]:
query = ''' 
SELECT
    purchase_date,
    purchase_id,
    client_id,
    client_age, 
    client_registration_age,
    IF(number_purchase=1,'NEW','OLD') AS client_category,
    promotion_name,
    category_name,
    partner_name,
    client_city, 
    city,
    revenue,
    quantity  
FROM
    (
    SELECT
        row_number(purchase_date) over w as number_purchase,
        purchase.purchase_date AS purchase_date ,
        purchase.purchase_id AS purchase_id ,
        purchase.client_id AS client_id ,
        dateDiff('year', client.birth_date, toDate(now( ))) AS client_age, 
        dateDiff('year', client.registration, toDate(now( ))) AS client_registration_age,
        promotion.promotion_name AS promotion_name ,
        promotion.category_name AS category_name,
        promotion.partner_name AS partner_name,
        client_city, 
        city,
        toFloat32(purchase.price*purchase.quantity) AS revenue,
        toInt32(purchase.quantity) as quantity
    FROM 
       project_variant_2.purchase purchase
    JOIN 
       project_variant_2.promotion promotion
    ON
       purchase.promotion_id = promotion.promotion_id
    JOIN 
        project_variant_2.client AS client 
    ON 
        client.client_id = purchase.client_id
    JOIN
        (SELECT DISTINCT 
            client_city_id, 
            client_city     
        FROM project_variant_2.city) AS client_city 
    ON 
        client.client_city_id = client_city.client_city_id
    JOIN 
        (SELECT DISTINCT 
            city_id,
            city     
        FROM project_variant_2.city) AS city_ 
    ON 
        purchase.city_id = city_.city_id   
                                
    WHERE purchase.status  = 1 AND purchase.purchase_date BETWEEN '2020-05-01' AND '2020-08-01'    
    window w as (partition by client_id order by purchase_date)
    )  
'''

In [6]:
df = ph.read_clickhouse(query=query, connection=connection_default)
df.head()

,purchase_date,purchase_id,client_id,client_age,client_registration_age,client_category,promotion_name,category_name,partner_name,client_city,city,revenue,quantity
0,2020-07-20,3002,1,26,1,NEW,campaign_2,Фрукты,"ООО Ширяева, Хохлова и Тимофеева",Вена,Новгород,660.0,3
1,2020-07-24,3626,1,26,1,OLD,campaign_2,Фрукты,"ООО Ширяева, Хохлова и Тимофеева",Вена,Ярославль,2280.0,8
2,2020-07-26,918,1,26,1,OLD,campaign_4,Прочее,KarpovCourses,Вена,Минск,615.0,3
3,2020-05-15,7362,2,23,3,NEW,campaign_1,Бакалея,Google,Щучинщина,Вена,180.0,9
4,2020-05-21,6702,2,23,3,OLD,campaign_4,Прочее,KarpovCourses,Щучинщина,Минск,465.0,3
